In [ ]:
!cp ../input/talibinstall/ta-lib-0.4.0-src.tar.gzh  ./ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz > null
!cd ta-lib && ./configure --prefix=/usr > null && make  > null && make install > null
!cp ../input/talibinstall/TA-Lib-0.4.21.tar.gzh TA-Lib-0.4.21.tar.gz
!pip install TA-Lib-0.4.21.tar.gz > null
!pip install ../input/talibinstall/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl >null

!rm -rf ./ta-lib
!rm  ./TA-Lib-0.4.21.tar.gz
!rm  ./ta-lib-0.4.0-src.tar.gz
!rm  ./null

import talib as ta

In [ ]:
import os, gc, warnings, random, datetime, traceback, joblib
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
import numpy.polynomial.hermite as Herm
import math
import tensorflow_probability as tfp

from tensorflow.keras import regularizers
from sklearn.preprocessing import RobustScaler
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K

In [ ]:
path_s = "../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/"
path_t = "../input/jpx-tokyo-stock-exchange-prediction/train_files/"
path_e = "../input/jpx-tokyo-stock-exchange-prediction/example_test_files/"

In [ ]:
prices1 = pd.read_csv(f"{path_s}stock_prices.csv") #2021
prices2 = pd.read_csv(f"{path_t}stock_prices.csv") #2017
prices3 = pd.read_csv(f"{path_t}secondary_stock_prices.csv")

prices1.shape , prices2.shape, prices3.shape

In [ ]:
prices2['Target'].isnull().sum(), prices1['Target'].isnull().sum(), prices3['Target'].isnull().sum()

In [ ]:
train2 = prices2.dropna(subset=['Target'])
train1 = prices1.dropna(subset=['Target'])

In [ ]:
train2['Target'].isnull().sum(), train1['Target'].isnull().sum()

In [ ]:
df = pd.concat([train2,train1])

In [ ]:
device = "GPU" 

In [ ]:
if device == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None
    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.TPUStrategy(tpu)
            print("TPU initialized")
        except: print("failed to initialize TPU")
    else: device = "GPU"

if device != "TPU": strategy = tf.distribute.get_strategy()
if device == "GPU": print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

In [ ]:
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
SEED = 2022
set_all_seeds(SEED)

In [ ]:
class Config:
    is_training = True
    output_dataset_path = "../input/models-for-jpx-aemlp-tf/"
config = Config()

In [ ]:
%%time
securities_codes = pd.read_csv("../input/jpx-cgkf-f98-tf/securities_codes.csv")
securities_codes_size = len(securities_codes) + 1
with tf.device("cpu"):
    securities_codes_lookup_layer = layers.IntegerLookup(max_tokens=securities_codes_size)
    securities_codes_lookup_layer.adapt(securities_codes)

In [ ]:
def prep_prices(prices):
    prices.fillna(0,inplace=True)
    return prices

In [ ]:
NDAYS = 180
lastdays = df[df["Date"]>=df.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
lastdays = pd.DataFrame(df.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]

In [ ]:
#simple units
hbar = 1.0
m    = 1.0
w    = 1.0

def hermite(x, n):
    xi             = np.sqrt(m*w/hbar)*x
    herm_coeffs    = np.zeros(n+1)
    herm_coeffs[n] = 1
    return Herm.hermval(xi, herm_coeffs)

def stationary_state(x,n):
    xi        = np.sqrt(m*w/hbar)*x
    prefactor = 1.0/math.sqrt(2.0**n * math.factorial(n)) * (m*w/(np.pi*hbar))**(0.25)
    psi       = prefactor * np.exp(- xi**2 / 2) * hermite(x,n)
    return psi

In [ ]:
from decimal import *
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

def adjust_features(df):
    df=df.copy()
    col='AdjustedClose'
    periods=[3,5,8,12,15,26,30,35,40,45,50,60, 100,200]
    for period in periods:
        df.loc[:,"Return_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].pct_change(period)
        df.loc[:,"MovingAvg_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].rolling(window=period).mean().values
        df.loc[:,"ExpMovingAvg_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].ewm(span=period,adjust=False).mean().values
        df.loc[:,"Volatility_{}Day".format(period)] = np.log(df[col]).groupby(df["SecuritiesCode"]).diff().rolling(period).std()
    return df

In [ ]:
def get_features(df, tr=True):    
    df = adjust_price(df)
    df = adjust_features(df)
    df = prep_prices(df)
    
    df['upper_Shadow']   = df['High'] - np.maximum(df['Close'], df['Open'])
    df['lower_Shadow']   = np.minimum(df['Close'], df['Open']) - df['Low'] 

    # The Golden Ratio Multiplier 
    df['GRM_0']    = (ta.MA(df['Close'], timeperiod=350, matype=0)) 
    df['GRM_1']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*1.6  
    df['GRM_2']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*2
    df['GRM_3']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*3

    df['Pi_Cycle'] = ta.MA(df['Close'], timeperiod=111, matype=0) 
    
    # Momentum
    df['RSI_14'] = ta.RSI(df['Close'], timeperiod=14)
    df['RSI_24'] = ta.RSI(df['Close'], timeperiod=24)
    df['RSI_42'] = ta.RSI(df['Close'], timeperiod=42)
    
    df['RSI1']   = df['RSI_14'].shift(-1) 
    df['RSI4']   = df['RSI_14'].shift(-4) 
    df['RSI7']   = df['RSI_14'].shift(-7) 
    df['RSI10']  = df['RSI_14'].shift(-10) 
    df['RSI13']  = df['RSI_14'].shift(-13) 
    df['RSI16']  = df['RSI_14'].shift(-16) 
    
    df['MACD_12'], df['macdsignal_12'], df['MACD_HIST_12'] = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
    df['MACD_48'], df['macdsignal_48'], df['MACD_HIST_48'] = ta.MACD(df['Close'], fastperiod=48, slowperiod=104, signalperiod=36)
    
    df['macdsignal1'] = df['macdsignal_12'].shift(-1)
    df['macdsignal4'] = df['macdsignal_12'].shift(-4)
    df['macdsignal7'] = df['macdsignal_12'].shift(-7)
    df['MACD_HIST1']  = df['MACD_HIST_12'].shift(-1) 
    df['MACD_HIST4']  = df['MACD_HIST_12'].shift(-4) 
    df['MACD_HIST7']  = df['MACD_HIST_12'].shift(-7) 
    df['ROCP']     = ta.ROCP(df['Open'])
    df['momentam'] = ta.MOM(df['Open'])
    df['CMO']      = ta.CMO(df['Open']) 
    df['PPO']      = ta.PPO(df['Open'])
    df['SAR']       = ta.SAR(df['High'], df['Low'], acceleration=0, maximum=0) 
    df['DI_minus']  = ta.MINUS_DI(df['High'], df['Low'],np.array(df.loc[:, 'Close']), timeperiod=14) 
    df['DI_minus1'] = df['DI_minus'].shift(-1) 
    df['DI_minus4'] = df['DI_minus'].shift(-4) 
    df['DI_minus7'] = df['DI_minus'].shift(-7)  
    df['adx']    = ta.ADX(df['High'], df['Low'],np.array(df.loc[:, 'Close']),timeperiod=14) 
    df['adx1']   = df['adx'].shift(-1) 
    df['adx4']   = df['adx'].shift(-4) 
    df['adx+1']  = df['adx'].shift(1) 
    df['adx7']   = df['adx'].shift(-7)
    df['DI_plus']   = ta.PLUS_DI(df['High'], df['Low'],np.array(df.loc[:, 'Close']), timeperiod=14) 
    df['DI_plus1']  = df['DI_plus'].shift(-1) 
    df['DI_plus4']  = df['DI_plus'].shift(-4) 
    df['DI_plus7']  = df['DI_plus'].shift(-7) 
    df['DI_plus10'] = df['DI_plus'].shift(-10)
    df['APO']      = ta.APO(df['Open'])
    df['APO1']     = df['APO'].shift(-1)
    df['APO4']     = df['APO'].shift(-4)
    df['APO7']     = df['APO'].shift(-7)
    df['ROCR100']  = ta.AD(df['High'], df['Low'], df['Close'], df['Volume'])
    df['OBV']      = ta.OBV(df['Close'], df['Volume'])
    df['ADOSC']    = ta.ADOSC(df['High'], df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)
    df['ATR']    = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['NATR']   = ta.NATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['Variance'] = ta.VAR(df['Close'], timeperiod=5, nbdev=1)
    df['CORREL']   = ta.CORREL(df['High'], df['Low'], timeperiod=15)
    df['TSF']      = ta.TSF(df['Close'], timeperiod=14) 
    df['TSF-14']   = ta.TSF(df['Close'], timeperiod=14).shift(-14)
    df['TSF-7']    = ta.TSF(df['Close'], timeperiod=14).shift(-7)
    df['ATR']    = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['NATR']   = ta.NATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    df['s_avg']      = df['SecuritiesCode'].apply(get_avg)
    df['qhm_115v']   = stationary_state(df['s_avg'], 115) 
    
    ema_set = [3,5,8,12,15,26,30,35,40,45,50,60, 100,200]
    # EMA
    for i in range(len(ema_set)):        
        sma = df['Close'].rolling(ema_set[i]).mean()
        ema = sma.ewm(span=ema_set[i], adjust=False).mean()
        df["EMA_C_%d"%(ema_set[i])] = ema
                                                 
        df = prep_prices(df)
    
    if tr:
        df = df.drop(['RowId','AdjustmentFactor','ExpectedDividend','SupervisionFlag','AdjustedClose'],axis=1)
    return df

In [ ]:
train_df = get_features(df)
groups   = pd.factorize(train_df.index)[0]
train    = train_df.reset_index(drop=True)

In [ ]:
securities_code = train.pop("SecuritiesCode")
y               = train.pop("Target")

In [ ]:
feat = train.columns
feat.shape[-1]

In [ ]:
del df
del train_df
del train

In [ ]:
def decode_function(record_bytes):
      return tf.io.parse_single_example(
      record_bytes,
      {
          "features": tf.io.FixedLenFeature([feat.shape[-1]], dtype=tf.float32),
          "securities_code": tf.io.FixedLenFeature([], dtype=tf.int64),
          "Target": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )
def preprocess(item):
    return (item["securities_code"], item["features"]), item["Target"]
def make_dataset(file_paths, batch_size=4096, mode="train"):
    ds = tf.data.TFRecordDataset(file_paths)
    ds = ds.map(decode_function)
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(batch_size*4*REPLICAS)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
def e_swish(beta=0.25):
    def beta_swish(x): return x*K.sigmoid(x)*(1+beta)
    return beta_swish

In [ ]:
def correlation(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr

In [ ]:
def correlationLoss(x,y, axis=-2):
    
    """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
    while trying to have the same mean and variance"""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr ) , dtype=tf.float32 )

In [ ]:
def sharpe_loss(X_train,y_pred):
    y_pred = tf.Variable(y_pred,dtype=tf.float64)
    port_ret = tf.reduce_sum(tf.multiply(X_train,y_pred),axis=1)
    s_ratio = K.mean(port_ret)/K.std(port_ret)
    
    return tf.math.exp(-s_ratio,  name='sharpe_loss')

In [ ]:
def build_model(hp, dim=139, fold=0):

    features_inputs = tf.keras.layers.Input(shape = (dim, ))
    securities_code_inputs = tf.keras.Input((1, ), dtype=tf.uint32)
    x0      =  tf.keras.layers.BatchNormalization()(features_inputs)

    weight = tf.Variable(tf.keras.backend.random_normal((dim, 1), stddev=hp.Float(f'weight_{fold}',1e-10, 0.09), dtype=tf.float32))
    var    = tf.Variable(tf.zeros((1,1), dtype=tf.float32))

    securities_code_x = (securities_code_inputs)
    securities_code_x = layers.Embedding(securities_codes_size, hp.Int(f'layer_emb{fold}',32, 512), input_length=1)(securities_code_x)
    securities_code_x = layers.Reshape((-1, ))(securities_code_x)
    securities_code_x = layers.Dense(hp.Int(f'layer_idx{fold}',32, 512), activation=e_swish(beta=hp.Float(f'e_idx_{fold}',0.15, 1, step=0.10)))(securities_code_x)
    
    encoder = tf.keras.layers.GaussianNoise(hp.Float(f'noise_{fold}',0.0001, 0.8))(x0)
    encoder = tf.keras.layers.Dense(hp.Int(f'layers{fold}_en0',32, 1024))(encoder)
    encoder = tf.keras.layers.Dense(hp.Int(f'layers{fold}_en1',32, 1024))(encoder)
    encoder = tf.keras.layers.Dense(hp.Int(f'layers{fold}_en2',32, 1024))(encoder)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation(e_swish(beta=hp.Float(f'e{fold}_en0',0.001, 1 )))(encoder)
    
    decoder = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_de0',0.001, 0.8))(encoder)
    decoder = tf.keras.layers.Dense(hp.Int(f'layers{fold}_de0',32, 1024), name='decoder')(decoder)

    x_ae = tf.keras.layers.Concatenate()([securities_code_x, decoder])
    x_ae = tf.keras.layers.Dense(hp.Int(f'layers{fold}_ae0',32, 1024))(x_ae)
    x_ae = tf.keras.layers.BatchNormalization()(x_ae)
    x_ae = tf.keras.layers.Activation(e_swish(beta=hp.Float(f'e{fold}_ae0',0.001, 1 )))(x_ae)
    x_ae = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_ae0',0.001, 0.8))(x_ae) 
    
    feature_x = tf.keras.layers.Concatenate()([x0, encoder])
    feature_x = tf.keras.layers.BatchNormalization()(feature_x)
    feature_x = tf.keras.layers.Dense(hp.Int(f'layers{fold}_fx0',32, 1024))(feature_x)
    feature_x = tf.keras.layers.Activation(e_swish(beta=hp.Float(f'e_fx0',0.001, 1 )))(feature_x)
    feature_x = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_fx0',0.001, 0.8))(feature_x)

    x = layers.Concatenate(axis=1)([securities_code_x, feature_x])

    x = layers.Dense(hp.Int(f'layers{fold}_x0',32, 1024), activation= e_swish(beta=hp.Float(f'e{fold}_x0',0.001, 1 )), kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_x0',0.0003, 0.9))(x)
    x = layers.Dense(hp.Int(f'layers{fold}_x1',32, 1024), activation= e_swish(beta=hp.Float(f'e{fold}_x1',0.001, 1 )), kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_x1',0.0003, 0.9))(x)
    x = layers.Dense(hp.Int(f'layers{fold}_x2',32, 1024), activation= e_swish(beta=hp.Float(f'e{fold}_x2',0.001, 1 )), kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dropout(hp.Float(f'dropout{fold}_x2',0.0003, 0.9))(x)

    mlp_out = layers.Dense(1, name ='mlp_out')(x)

    model  = tf.keras.Model(inputs=[securities_code_inputs, features_inputs], outputs=[decoder, mlp_out])

    loss_out = tf.add(tf.matmul(features_inputs,weight), tf.math.reduce_sum(weight*var))
    tf.compat.v1.losses.add_loss(loss_out)
  
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float(f'lr_adam{fold}',1e-2, 1e-12)),
                  loss = {'decoder': [tf.keras.losses.CosineSimilarity(axis=-2), 
                                      tf.keras.losses.MeanSquaredError(), 
                                      correlationLoss],         
                          
                          'mlp_out' : [tf.keras.losses.MeanSquaredError(), sharpe_loss],
                         },
                  metrics = {'decoder': [tf.keras.metrics.CosineSimilarity(name='cosine', axis=-2),
                                         tf.keras.metrics.MeanAbsoluteError(name="mae"), 
                                         correlation, 
                                         tf.keras.metrics.RootMeanSquaredError(name='rmse')], 
                             
                             'mlp_out' : [tf.keras.metrics.CosineSimilarity(name='cosine', axis=-2),
                                          tf.keras.metrics.MeanAbsoluteError(name="mae"), 
                                          correlation, 
                                          tf.keras.metrics.RootMeanSquaredError(name='rmse')],
                            },
                 ) 
    return model

In [ ]:
hp = pd.read_pickle('../input/hp-jpx-aemlp/hp_aempp_tf_139f_5f_tf.pkl')
tf.keras.utils.plot_model(build_model(hp), show_shapes=True, expand_nested=True, show_dtype=True)

In [ ]:
def get_lr_callback(batch_size = 8):
    lr_start   = 3e-3
    lr_max     = 1.25e-10 * REPLICAS * batch_size
    lr_min     = 3e-12
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.98
    def lrfn(epoch):
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        else: lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        return lr
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

In [ ]:
%%time
gc.collect()
models = []

fold = [0,1,2,3,4]
for i in range(len(fold)):
    train_path = f"../input/jpx-cgkf-f98-tf/fold_{fold[i]}_train.tfrecords"
    valid_path = f"../input/jpx-cgkf-f98-tf/fold_{fold[i]}_test.tfrecords"
    valid_ds   = make_dataset([valid_path], mode="valid")
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{fold[i]}.tf", monitor="val_mlp_out_loss", mode="min", save_best_only=True, save_weights_only=True)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_mlp_out_loss', patience=7, mode='min', restore_best_weights=True)
    K.clear_session()
    with strategy.scope(): model = build_model(hp, dim=feat.shape[-1], fold=fold[i])
    if config.is_training:
        train_ds = make_dataset([train_path])
        history = model.fit(train_ds, epochs=100, validation_data=valid_ds, callbacks=[checkpoint, 
                                                                                       early_stop,
                                                                                       get_lr_callback(batch_size = 4096*REPLICAS)])
        gc.collect()
        model.load_weights(f"model_{fold[i]}.tf")
        
        for metric in ["mlp_out_loss", "mlp_out_rmse", "mlp_out_cosine", "mlp_out_correlation"]:
            pd.DataFrame(history.history, columns=[metric, f"val_{metric}"]).plot()
            plt.title(metric.upper())
            plt.show()
            gc.collect()
        
    else:
        model.load_weights(f"{config.output_dataset_path}model_{fold[i]}.tf")
    y_vals = []
    for _, y in valid_ds:
        y_vals += list(y.numpy().reshape(-1))
    y_val = np.array(y_vals)
    pearson_score = stats.pearsonr((model.predict(valid_ds)[-1].reshape(-1)), y_val)[0]
    models.append(model)
    print(f"Pearson Score: {pearson_score}")
    gc.collect()

In [ ]:
def preprocess_test(securities_code, feature):
    return (securities_code, feature), 0

def make_test_dataset(feature, securities_code, batch_size=4096*REPLICAS):
    ds = tf.data.Dataset.from_tensor_slices(((securities_code, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)[-1].reshape(-1)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()                   # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    
    prices_df                        = get_features(prices, tr=False)
    sample_prediction["Prediction"]  = inference(models, make_test_dataset(prices_df[feat], prices['SecuritiesCode']))
    sample_prediction["rate"]        = sample_prediction["Prediction"]
    sample_prediction.sort_values(by = "rate", ascending=False, inplace=True)
    sample_prediction.Rank           = np.arange(0,2000)
    sample_prediction.sort_values(by = "SecuritiesCode", ascending=True, inplace=True)
    submission                       = sample_prediction[["Date","SecuritiesCode","Rank"]]
    display(submission)
    env.predict(submission)  

# PREVIOUS LEVEL
* [JPX(0/2): AEMLP-GKF: Natapong Nitarach](https://www.kaggle.com/code/natnitarach/jpx-aemlp-gkf)

# NEXT LEVEL
* [JPX(2/2): LSTM+GRU-TF: Natapong Nitarach](https://www.kaggle.com/natnitarach/jpx-2-2-lstm-gru-tf/)

# CHANGE UNIVERSE
* [Crypto Forecasting(1/1) : SAEMLP-QHO: Natapong Nitarach](https://www.kaggle.com/code/natnitarach/crypto-forecasting-1-1-saemlp-qho?scriptVersionId=87744356)